# Datorseenede projekt 
#### Projektet i kursen datorseende

### Installationer
Installerar paket som behövs för exekveringen av koden.

In [7]:
%pip install split-folders
%pip install --upgrade ultralytics
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
   - -------------------------------------- 8.1/204.2 MB 46.0 MB/s eta 0:00:05
   --- ------------------------------------ 17.6/204.2 MB 44.4 MB/s eta 0:00:05
   ----- ---------------------------------- 27.0/204.2 MB 43.9 MB/s eta 0:00:05
   ------ --------------------------------- 35.7/204.2 MB 42.7 MB/s eta 0:00:04
   -------- ------------------------------- 44.6/204.2 MB 43.0 MB/s eta 0:00:04
   ---------- ----------------------------- 53.0/204.2 MB 42.2 MB/s eta 0:00:04
   ----------- ---------------------------- 61.1/204.2 MB 41.4 MB/s eta 0:00:04
   ------------- -------------------------- 70.3/204.2 MB 41.9 MB/s eta 0:00:04
   --------------- ------------------------ 79.4/204.2 MB 41.9 MB/s eta 0:00:03
   ----------------- ---------------------- 88.1/204.2 MB 41.6 MB/s eta 0:00:03
   ------------------- -------------------- 97.3/204.2 MB 41.7 M

### Spjälka data 
Spjälker data för att kunna användas i yolo modellen. Data spjälks till test, train och val.

In [1]:
import torch
import torchvision
print(torch.__version__)  # Ska vara 2.6.0+cu126
print(torchvision.__version__)  # Ska ha +cu126, inte +cpu
print(torch.cuda.is_available())  # Ska vara True


2.6.0+cu126
0.21.0+cu126
True


## Utför träningen 
Träna en yolov11 model med datan som vi har, med passande parametrar för egen dator att köra.

In [1]:
import splitfolders

input_folder = "./input-data"
output_folder = "./dataset"

splitfolders.ratio(input_folder,output = output_folder, seed=42, ratio=(0.8,0.1,0.1),group_prefix=None)

Copying files: 3913 files [00:33, 117.63 files/s]


In [2]:
from ultralytics import YOLO

# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolo11n.pt")


results = model.train(
    data="config.yaml",        # Din träningsdata
    project="./output",         # Utmatningsmapp
    device='cuda:0',            # Träna på GPU (ändra om du vill använda CPU)
    epochs=10,                 # Antal epoker
    batch=16,                   # Låt YOLO välja den bästa batch-storleken baserat på GPU-minne
    imgsz=540,                  # Bildstorlek, 640x640 bör vara bra för din GPU
    amp=True,                   # Aktivera Automatic Mixed Precision för snabbare träning och mindre minne              # Spara modellen var 5:e epok
    optimizer='adam',           # Använd Adam optimizer för bättre konvergens
    workers=12,                 # Antal arbetare för dataladdning, öka om du har fler CPU-kärnor       
    name="test",          # Namn på träningssessionen
    patience=20,                # Stoppar om ingen förbättring efter 10 epochs
    exist_ok=True,              # Om du vill skriva över tidigare resultat
)
# Evaluate the model's performance on the validation set
results = model.val()

# Export the model to ONNX format
success = model.export(format="onnx")

Ultralytics 8.3.88  Python-3.11.11 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1660, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=config.yaml, epochs=10, time=None, patience=20, batch=16, imgsz=540, save=True, save_period=-1, cache=False, device=cuda:0, workers=12, project=./output, name=test, exist_ok=True, pretrained=True, optimizer=adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

train: Scanning E:\Programming\cv-25-project\dataset\train\labels... 745 images, 1281 backgrounds, 0 corrupt: 100%|██████████| 2024/2024 [00:03<00:00, 639.98it/s]

train: WARNING  E:\Programming\cv-25-project\dataset\train\images\460a9aad-Giant_Slalom_Racer.jpg: corrupt JPEG restored and saved


train: New cache created: E:\Programming\cv-25-project\dataset\train\labels.cache


val: Scanning E:\Programming\cv-25-project\dataset\val\labels... 11 images, 242 backgrounds, 0 corrupt: 100%|██████████| 253/253 [00:00<00:00, 388.94it/s]


...


10 epochs completed in 0.136 hours.
Optimizer stripped from output\test\weights\last.pt, 5.5MB
Optimizer stripped from output\test\weights\best.pt, 5.5MB

Validating output\test\weights\best.pt...
Ultralytics 8.3.88  Python-3.11.11 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1660, 6144MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  4.61it/s]


                   all        253         15     0.0856       0.25     0.0784     0.0307
                 skier          7         10      0.133        0.3     0.0547     0.0132
           snowboarder          4          5     0.0382        0.2      0.102     0.0483
Speed: 0.2ms preprocess, 3.3ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to output\test
Ultralytics 8.3.88  Python-3.11.11 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1660, 6144MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning E:\Programming\cv-25-project\dataset\val\labels.cache... 11 images, 242 backgrounds, 0 corrupt: 100%|██████████| 253/253 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:02<00:00,  5.66it/s]


                   all        253         15     0.0823       0.25      0.072     0.0298
                 skier          7         10      0.126        0.3      0.051     0.0123
           snowboarder          4          5     0.0382        0.2     0.0929     0.0473
Speed: 0.7ms preprocess, 5.2ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to output\test
Ultralytics 8.3.88  Python-3.11.11 torch-2.6.0+cu126 CPU (AMD Ryzen 7 2700 Eight-Core Processor)

PyTorch: starting from 'output\test\weights\best.pt' with input shape (1, 3, 544, 544) BCHW and output shape(s) (1, 6, 6069) (5.2 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.48...
ONNX: export success  2.7s, saved as 'output\test\weights\best.onnx' (10.0 MB)

Export complete (3.1s)
Results saved to E:\Programming\cv-25-project\output\test\weights
Predict:         yolo predict task=detect model=output\test\weights\best.onnx imgsz=544  
Validate:        yolo val task=detect mo